# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
!pip install langchain langchain-openai python-dotenv

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [9]:
llm = ChatOpenAI(temperature=0.7)

In [10]:
prompt = ChatPromptTemplate.from_template('What is the price of a {product}?')

In [11]:

chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [12]:
product = 'tesla s car'
chain.run(product)

/tmp/ipython-input-1253719449.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'The price of a Tesla Model S varies depending on the specific model, options, and region. The starting price for the Tesla Model S Long Range is around $79,990, while the Performance model starts at around $99,990. Additional options, upgrades, and packages can increase the price further.'

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    'What is the price of a {product}?'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Given the following price information: {price_info}. Could you give me the different models of tesla car that fit within that price range or are relevant to that information?'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The price of a Tesla Model S car can vary depending on the specific model, features, and any additional options added. As of 2021, the starting price for a Tesla Model S is around $79,990. However, this price can increase significantly with added features, upgrades, and customizations.
Certainly! Within the price range of around $79,990, there are a few different Tesla models that fall into that range or are relevant to the information provided:

1. Tesla Model S: The starting price for the standard Tesla Model S is around $79,990. This model comes with a range of features and options that can be customized to suit individual preferences.

2. Tesla Model 3: The Tesla Model 3 is another popular electric car from Tesla that falls within a similar price range. The starting price for the Model 3 is lower than the Model S, but with added features and upgrades, it can reach a similar price point.

3. Tesla Model Y: The Tesla Model Y is a crosso

"Certainly! Within the price range of around $79,990, there are a few different Tesla models that fall into that range or are relevant to the information provided:\n\n1. Tesla Model S: The starting price for the standard Tesla Model S is around $79,990. This model comes with a range of features and options that can be customized to suit individual preferences.\n\n2. Tesla Model 3: The Tesla Model 3 is another popular electric car from Tesla that falls within a similar price range. The starting price for the Model 3 is lower than the Model S, but with added features and upgrades, it can reach a similar price point.\n\n3. Tesla Model Y: The Tesla Model Y is a crossover SUV that is also within the price range of around $79,990. It offers ample space and features similar to the Model S and Model 3.\n\nIt's important to note that prices may vary based on location, specific configurations, and any additional options chosen. Customers can customize their Tesla vehicles to fit their preference

**Repeat the above twice for different products**

In [22]:
product2 = 'Queen Size Sheet Set'
chain.run(product2)

'The price of a Queen Size Sheet Set can vary depending on the brand, material, and quality. On average, a Queen Size Sheet Set can range from $30 to $150 or more.'

In [23]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    'What is the price of a {product2}?'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [24]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Given the following price information: {price_info}. Could you give me the different models of {product2}  that fit within that price range or are relevant to that information?'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
from langchain.chains import SequentialChain


In [29]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(temperature=0.9)

# -----------------------
# CHAIN 1 - Get price
# -----------------------
first_prompt = ChatPromptTemplate.from_template(
    "What is the price of a {product2}?"
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="price_info"   # <--- RENOMBRAMOS OUTPUT
)

# -----------------------
# CHAIN 2 - Suggest models
# -----------------------
second_prompt = ChatPromptTemplate.from_template(
    "Given the following price information: {price_info}. "
    "Could you give me the different models of {product2} "
    "that fit within that price range or are relevant to that information?"
)

chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="final_answer"  # <--- RENOMBRAMOS OUTPUT
)

# -----------------------
# PIPELINE
# -----------------------
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["product2"],
    output_variables=["final_answer"],
    verbose=True
)

# -----------------------
# RUN
# -----------------------
product2 = "Queen Size Sheet Set"

response = overall_chain({"product2": product2})
print(response["final_answer"])


/tmp/ipython-input-3597574645.py:51: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = overall_chain({"product2": product2})




> Entering new SequentialChain chain...

> Finished chain.
Sure, here are some Queen Size Sheet Sets that fall within the price range mentioned:

1. Mainstays Jersey Knit Sheet Set - $30
2. AmazonBasics Microfiber Sheet Set - $40
3. Utopia Bedding 4 Piece Sheet Set - $50
4. Mellanni Bed Sheet Set - $30-$60
5. HC Collection Bed Sheet & Pillowcase Set - $40-$70
6. Nestl Bedding 4 Piece Sheet Set - $50-$80
7. Top Split King Royal Collection 1900 Thread Count Sheet Set - $90-$120
8. Pure Beech Jersey Knit Modal Sheet Set - $100-$150

These are just a few examples of Queen Size Sheet Sets that fall within the price range mentioned. Prices may vary depending on the brand, material, thread count, and any additional features.


In [26]:
overall_simple_chain.run(product2)



> Entering new SimpleSequentialChain chain...
The price of a Queen Size Sheet Set can vary depending on the brand, material, and quality. On average, a Queen Size Sheet Set can range from $30 to $150 or more.
Based on the price range provided, it does not appear that any Tesla car models currently fit within the $30 to $150 price range for a Queen Size Sheet Set. Tesla cars are typically luxury electric vehicles with prices starting in the tens of thousands of dollars and going up from there. Some Tesla models that are relevant to the information provided include:

1. Tesla Model 3 - The starting price for a Tesla Model 3 is around $40,000, which is above the $30 minimum price range mentioned.
2. Tesla Model Y - The starting price for a Tesla Model Y is around $50,000, also above the $30 minimum price range.
3. Tesla Model S - The starting price for a Tesla Model S is around $70,000, well above the $30 minimum price range.
4. Tesla Model X - The starting price for a Tesla Model X is 

'Based on the price range provided, it does not appear that any Tesla car models currently fit within the $30 to $150 price range for a Queen Size Sheet Set. Tesla cars are typically luxury electric vehicles with prices starting in the tens of thousands of dollars and going up from there. Some Tesla models that are relevant to the information provided include:\n\n1. Tesla Model 3 - The starting price for a Tesla Model 3 is around $40,000, which is above the $30 minimum price range mentioned.\n2. Tesla Model Y - The starting price for a Tesla Model Y is around $50,000, also above the $30 minimum price range.\n3. Tesla Model S - The starting price for a Tesla Model S is around $70,000, well above the $30 minimum price range.\n4. Tesla Model X - The starting price for a Tesla Model X is around $80,000, also well above the $30 minimum price range.\n\nIt is important to note that Tesla cars are high-end luxury vehicles, and their prices reflect that.'

## SequentialChain

In [30]:
from langchain.chains import SequentialChain

In [31]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following review into English: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="translated_review"
                    )

In [32]:
second_prompt = ChatPromptTemplate.from_template(
    'Summarize the review:{translated_review}'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='english_summarize' #give a name to this output
                    )


In [33]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'Translate the following English summary into Spanish: {english_summarize}'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='spanish_summary'
                      )

In [34]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
      'Based on the English summary: {english_summarize} and the Spanish summary: {spanish_summary}, provide a follow up message.'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='followup_message'
                     )

In [35]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['english_summarize', 'spanish_summary', 'followup_message'],
    verbose=True
)

In [36]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'english_summarize': 'The reviewer is dissatisfied with the taste of the product and finds the foam to be strange and not long-lasting. They suspect that the product may be an old batch or counterfeit, as they have had a better experience with the same product purchased in stores.',
 'spanish_summary': 'El revisor está insatisfecho con el sabor del producto y encuentra que la espuma es extraña y no duradera. Sospechan que el producto puede ser de una partida antigua o falsificado, ya que han tenido una mejor experiencia con el mismo producto comprado en tiendas.',
 'followup_message': 'I would recommend reaching out to the manufacturer or seller of the product to express your concerns and inquire about the possibility of receiving a replacement or refund. It is important to address issues like this to ensure t

**Repeat the above twice for different products or reviews**

In [37]:
import textwrap
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

# ======================================================
# 1️⃣ CHAIN 1 — Categorizar la pregunta psicológica
# ======================================================
prompt_1 = ChatPromptTemplate.from_template(
    "Categorize the following psychology question into one domain "
    "(cognition, emotion, personality, social, or biases). "
    "Question: {question}"
)

chain_1 = LLMChain(
    llm=llm,
    prompt=prompt_1,
    output_key="psychology_domain"
)

# ======================================================
# 2️⃣ CHAIN 2 — Explicar el concepto
# ======================================================
prompt_2 = ChatPromptTemplate.from_template(
    "Explain the psychological concept related to this domain: {psychology_domain}. "
    "User question: {question}."
)

chain_2 = LLMChain(
    llm=llm,
    prompt=prompt_2,
    output_key="concept_explanation"
)

# ======================================================
# 3️⃣ CHAIN 3 — Aplicarlo a un ejemplo real
# ======================================================
prompt_3 = ChatPromptTemplate.from_template(
    "Provide a real-world example that illustrates the concept: {concept_explanation}."
)

chain_3 = LLMChain(
    llm=llm,
    prompt=prompt_3,
    output_key="real_example"
)

# ======================================================
# 4️⃣ CHAIN 4 — Dar recomendaciones prácticas
# ======================================================
prompt_4 = ChatPromptTemplate.from_template(
    "Based on the example '{real_example}', give practical psychological advice "
    "or strategies someone could use in real life."
)

chain_4 = LLMChain(
    llm=llm,
    prompt=prompt_4,
    output_key="practical_advice"
)

# ======================================================
# FULL SEQUENTIAL CHAIN
# ======================================================
psychology_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["question"],
    output_variables=["concept_explanation", "real_example", "practical_advice"],
    verbose=True
)

# ======================================================
# QUESTIONS TO TEST
# ======================================================
questions = [
    "Why do people procrastinate?",
    "What is cognitive dissonance?",
    "How does social anxiety develop?",
    "What is emotional regulation?",
    "Why do people conform in groups?"
]

# ======================================================
# EXACT FORMAT REQUESTED
# ======================================================
for q in questions:
    print("\n======================================")
    print(f"📌 USER QUESTION:\n{q}")
    print("--------------------------------------")

    result = psychology_chain({"question": q})

    final_text = (
        f"Concept Explanation:\n{result['concept_explanation']}\n\n"
        f"Real Example:\n{result['real_example']}\n\n"
        f"Practical Advice:\n{result['practical_advice']}"
    )

    print("🧠 PSYCHOLOGY ANALYSIS:\n")
    print(textwrap.fill(final_text, width=90))
    print("======================================\n")



📌 USER QUESTION:
Why do people procrastinate?
--------------------------------------


> Entering new SequentialChain chain...

> Finished chain.
🧠 PSYCHOLOGY ANALYSIS:

Concept Explanation: Procrastination is a common behavior that is often influenced by
cognitive processes. One psychological concept related to cognition that can help explain
procrastination is the concept of cognitive dissonance. Cognitive dissonance refers to the
discomfort that arises when an individual holds contradictory beliefs or engages in
behaviors that are inconsistent with their attitudes or values.  When individuals
procrastinate, they may experience cognitive dissonance because they know they should be
completing a task, but they also feel a strong desire to avoid it in favor of more
immediately rewarding activities. This conflict between what they should be doing and what
they want to do can create discomfort, leading them to put off the task even further.
Additionally, cognitive biases such as present 

## Router Chain

In [38]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [39]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [40]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [41]:
llm = ChatOpenAI(temperature=0)

In [42]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [43]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [44]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [46]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [47]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [48]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [49]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, having DNA in every cell allows for genetic diversity and variation within an organism, which is important for adaptation and evolution. This genetic diversity allows for different cells to specialize and perform specific functions within the body, contributing to the overall health and functioning of the organism.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

# **ROUTER CHAIN WITH ECONOMIC TOPICS**

In [50]:
# -----------------------------------------------------------
# 1. IMPORTS
# -----------------------------------------------------------
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# -----------------------------------------------------------
# 2. BASE LLM
# -----------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [51]:
# -----------------------------------------------------------
# 3. DESTINATION PROMPTS: ECONOMIC TOPICS
# -----------------------------------------------------------

prompt_infos = [
    {
        "name": "stock_market",
        "description": "For questions about stocks, ETFs, indices, and trading strategies.",
        "prompt_template": """
You are a financial analyst specialized in the stock market.
Explain clearly using data-driven logic.
If the question is not about the stock market, respond: "I only answer stock market questions."

User question:
{input}
""",
    },
    {
        "name": "cryptocurrencies",
        "description": "For questions about Bitcoin, Ethereum, blockchain, DeFi, and altcoins.",
        "prompt_template": """
You are a blockchain and cryptocurrency expert.
Explain crypto concepts clearly and avoid financial advice.
If the question is not about crypto, respond: "I only answer cryptocurrency questions."

User question:
{input}
""",
    },
    {
        "name": "value_investing",
        "description": "For questions about fundamentals, Warren Buffett, intrinsic value, and long-term investing.",
        "prompt_template": """
You are a value investing specialist trained in principles used by Warren Buffett and Benjamin Graham.
Explain conservative, fundamental-analysis approaches.
If the question is not about value investing, respond: "I only answer value investing questions."

User question:
{input}
""",
    },
    {
        "name": "forex",
        "description": "For questions about currency pairs, central banks, and foreign exchange trading.",
        "prompt_template": """
You are a Forex markets analyst.
Explain FX concepts like interest rates, monetary policy, and currency dynamics.
If the question is not about Forex trading, respond: "I only answer Forex questions."

User question:
{input}
""",
    },
]


In [52]:
# -----------------------------------------------------------
# 4. CREATE DESTINATION CHAINS
# -----------------------------------------------------------
destination_chains = {}

for info in prompt_infos:
    name = info["name"]
    prompt_template = info["prompt_template"]

    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# -----------------------------------------------------------
# 5. ROUTER PROMPT
# -----------------------------------------------------------

destinations_str = "\n".join(
    [f"{p['name']}: {p['description']}" for p in prompt_infos]
)

# The router_template needs to output JSON for RouterOutputParser
router_template_economic = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_prompt = PromptTemplate(
    template=router_template_economic.format(destinations=destinations_str),
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [53]:
# -----------------------------------------------------------
# 6. MULTI-PROMPT ROUTER CHAIN
# -----------------------------------------------------------
router_system = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            "No domain detected. I am the default assistant. Question: {input}"
        ),
        verbose=True
    ),
    verbose=True
)

In [54]:
# -----------------------------------------------------------
# 7. TEST ROUTER WITH EXAMPLES
# -----------------------------------------------------------
import textwrap

questions = [
    "Is Nvidia overvalued at the current P/E ratio?",
    "Will Bitcoin reach a new ATH this cycle?",
    "How did Warren Buffett calculate intrinsic value?",
    "What affects EUR/USD volatility?",
    "Tell me a joke about finance."
]

for q in questions:
    print("\n======================================")
    print(f"📌 USER QUESTION:\n{q}")
    print("--------------------------------------")

    result = router_system.run(q)

    print("🧠 ROUTED ANSWER:\n")
    print(textwrap.fill(result, width=90))
    print("======================================\n")



📌 USER QUESTION:
Is Nvidia overvalued at the current P/E ratio?
--------------------------------------


> Entering new MultiPromptChain chain...
stock_market: {'input': 'Is Nvidia overvalued at the current P/E ratio?'}
> Finished chain.
🧠 ROUTED ANSWER:

To determine if Nvidia is overvalued at its current P/E (Price-to-Earnings) ratio, we need
to analyze several factors, including its historical P/E ratio, the industry average,
growth prospects, and overall market conditions.  1. **Current P/E Ratio**: As of the
latest data, Nvidia's P/E ratio is significantly higher than the average for the
semiconductor industry, which typically ranges from 15 to 25. For example, if Nvidia's P/E
ratio is around 50, it suggests that investors are willing to pay a premium for its
earnings.  2. **Historical P/E Ratio**: Comparing Nvidia's current P/E ratio to its
historical averages can provide insight. If Nvidia's historical P/E has been around 30, a
current P/E of 50 may indicate overvaluation, assu

In [55]:
router_system.run('what are the market cap of Palantir')



> Entering new MultiPromptChain chain...
stock_market: {'input': 'What is the market cap of Palantir?'}
> Finished chain.


'As of my last update in October 2023, Palantir Technologies Inc. (ticker: PLTR) had a market capitalization that fluctuated around $15 billion to $20 billion. However, market caps can change frequently due to stock price movements. To get the most accurate and up-to-date market cap, I recommend checking a reliable financial news website or stock market platform for the latest stock price and market capitalization.'

In [56]:
router_system.run('Explain me ethereum')




> Entering new MultiPromptChain chain...
cryptocurrencies: {'input': 'Explain Ethereum and its significance in the cryptocurrency space.'}
> Finished chain.


"Ethereum is a decentralized, open-source blockchain platform that enables developers to build and deploy smart contracts and decentralized applications (dApps). Launched in 2015 by Vitalik Buterin and others, Ethereum introduced the concept of programmable blockchain, which allows for more complex transactions and interactions than Bitcoin's primarily transactional model.\n\n### Key Concepts of Ethereum:\n\n1. **Smart Contracts**: These are self-executing contracts with the terms of the agreement directly written into code. They automatically enforce and execute the terms when predetermined conditions are met, reducing the need for intermediaries.\n\n2. **Decentralized Applications (dApps)**: These are applications that run on the Ethereum blockchain rather than being hosted on centralized servers. They can range from games to financial services and social networks, all operating in a decentralized manner.\n\n3. **Ether (ETH)**: This is the native cryptocurrency of the Ethereum networ

In [57]:
router_template = f"""
You are a strict routing agent.

Choose ONLY one expert name from the list below.
Respond with EXACTLY one of the following strings:

- stock_market
- cryptocurrencies
- value_investing
- forex

DO NOT add explanations.
DO NOT write anything else.
DO NOT use capital letters.
DO NOT add punctuation.

Here are the available experts:
{destinations_str}

User question:
{{input}}
"""


In [58]:
router_chain = LLMChain(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=3),
    prompt=router_prompt,
    verbose=True
)


In [59]:
router_system.run('Is Nvidia overvalued at the current P/E ratio?')



> Entering new MultiPromptChain chain...
stock_market: {'input': 'Is Nvidia overvalued at the current P/E ratio?'}
> Finished chain.


"To determine if Nvidia is overvalued at its current P/E (Price-to-Earnings) ratio, we need to analyze several factors, including its historical P/E ratio, industry averages, growth prospects, and overall market conditions.\n\n1. **Current P/E Ratio**: As of the latest data, Nvidia's P/E ratio is significantly higher than the average for the semiconductor industry, which typically ranges from 15 to 25. For example, if Nvidia's P/E ratio is around 50, it suggests that investors are willing to pay a premium for its earnings.\n\n2. **Historical P/E Ratio**: Comparing Nvidia's current P/E ratio to its historical averages can provide insight. If Nvidia's historical P/E has been around 30, a current P/E of 50 may indicate overvaluation, assuming no significant changes in growth prospects.\n\n3. **Growth Prospects**: Nvidia has been a leader in AI and gaming technologies, which has driven substantial revenue growth. If analysts project high growth rates (e.g., 20-30% annually), a higher P/E r

# **FULL COPY–PASTE ROUTER CHAIN (with verbose + wrapped output)**

In [60]:
# -----------------------------------------------------------
# 1. IMPORTS
# -----------------------------------------------------------
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain_openai import ChatOpenAI
import textwrap

# -----------------------------------------------------------
# 2. BASE LLM
# -----------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)




In [61]:
# -----------------------------------------------------------
# 3. DESTINATION PROMPTS (TRAINING SPECIALISTS)
# -----------------------------------------------------------

prompt_infos = [
    {
        "name": "crossfit",
        "description": "For CrossFit, HIIT, functional fitness, and mixed modal workouts.",
        "prompt_template": """
You are an elite CrossFit coach.
Specialized in WOD programming, metabolic conditioning, gymnastics,
barbell cycling, and functional fitness.

If the question is not about CrossFit, reply:
"I only answer CrossFit questions."

User question:
{input}
""",
    },
    {
        "name": "sprinting",
        "description": "For athletics training in 100m–400m sprinting, speed mechanics, and power.",
        "prompt_template": """
You are a high-performance athletics sprint coach.
Expert in 100m and 400m training cycles, acceleration technique,
speed endurance, block starts, and stride efficiency.

If the question is not about sprinting (100–400m), reply:
"I only answer sprinting questions."

User question:
{input}
""",
    },
    {
        "name": "swimming",
        "description": "For competitive swimming technique and conditioning.",
        "prompt_template": """
You are an elite competitive swimming coach.
Specialized in freestyle, butterfly, breaststroke, backstroke,
turn technique, pacing, and breathing mechanics.

If the question is not about swimming, reply:
"I only answer swimming questions."

User question:
{input}
""",
    },
]


In [62]:
# -----------------------------------------------------------
# 4. BUILD DESTINATION CHAINS (WITH VERBOSE)
# -----------------------------------------------------------
destination_chains = {}

for info in prompt_infos:
    prompt = ChatPromptTemplate.from_template(info["prompt_template"])
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)   # VERBOSE ENABLED
    destination_chains[info["name"]] = chain

In [63]:
# -----------------------------------------------------------
# 5. ROUTER PROMPT
# -----------------------------------------------------------
destinations_str = "\n".join(
    [f"{p['name']}: {p['description']}" for p in prompt_infos]
)

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_prompt = PromptTemplate(
    template=MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str),
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [64]:
# -----------------------------------------------------------
# 6. MULTIPROMPT ROUTER SYSTEM (WITH VERBOSE)
# -----------------------------------------------------------
default_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(
        "I am the default assistant. No matching training category. Question: {input}"
    ),
    verbose=True
)

router_system = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True   # VERBOSE HERE
)



In [65]:
# -----------------------------------------------------------
# 7. TEST QUESTIONS
# -----------------------------------------------------------
questions = [
    "Give me a 12-minute AMRAP that focuses on shoulders and core.",
    "How can I improve my 100m acceleration out of the blocks?",
    "How do I become more efficient in freestyle breathing?",
    "What is the best way to build lactate tolerance for 400m?",
    "I want to improve my butterfly kick technique.",
]

for q in questions:
    print("\n======================================")
    print(f"📌 USER QUESTION:\n{q}")
    print("--------------------------------------")

    result = router_system.run(q)

    print("🧠 ROUTED ANSWER:\n")
    print(textwrap.fill(result, width=90))
    print("======================================\n")


📌 USER QUESTION:
Give me a 12-minute AMRAP that focuses on shoulders and core.
--------------------------------------


> Entering new MultiPromptChain chain...
crossfit: {'input': 'Give me a 12-minute AMRAP workout that focuses on shoulders and core.'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: 
You are an elite CrossFit coach.
Specialized in WOD programming, metabolic conditioning, gymnastics,
barbell cycling, and functional fitness.

If the question is not about CrossFit, reply:
"I only answer CrossFit questions."

User question:
Give me a 12-minute AMRAP workout that focuses on shoulders and core.


> Finished chain.

> Finished chain.
🧠 ROUTED ANSWER:

Here's a 12-minute AMRAP (As Many Rounds As Possible) workout that focuses on shoulders
and core:  **12-Minute AMRAP:**  1. **Push Press** (95/65 lbs) - 10 reps 2. **Plank
Shoulder Taps** - 20 reps (10 per side) 3. **Dumbbell Front Raises** (15/10 lbs) - 10 reps
4. **Russian Twists** - 20 reps (10 per side) 5